In [1]:
from pyspark.sql import SparkSession

# SparkSession 생성
spark = SparkSession.builder.appName("BankMarketing-EDA").getOrCreate()

In [4]:
file_path = './ratings.csv'

ratings = spark.read.csv(file_path, inferSchema = True, header = True)
ratings.printSchema()

root
 |-- book_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- rating: integer (nullable = true)



In [5]:

ratings.show(5)

# NULL이나 이상치 제거 (옵션)
ratings = ratings.dropna()

+-------+-------+------+
|book_id|user_id|rating|
+-------+-------+------+
|      1|    314|     5|
|      1|    439|     3|
|      1|    588|     5|
|      1|   1169|     4|
|      1|   1185|     4|
+-------+-------+------+
only showing top 5 rows



In [6]:
train, test = ratings.randomSplit([0.8, 0.2], seed=42)

In [7]:
from pyspark.ml.recommendation import ALS

als = ALS(
    userCol="user_id",
    itemCol="book_id",
    ratingCol="rating",
    nonnegative=True,
    coldStartStrategy="drop",  # NaN 예측 제거
    implicitPrefs=False,       # 명시적 피드백 사용
    rank=10,
    maxIter=10,
    regParam=0.1
)

model = als.fit(train)

In [8]:
from pyspark.ml.evaluation import RegressionEvaluator

predictions = model.transform(test)

evaluator = RegressionEvaluator(
    metricName="rmse", 
    labelCol="rating", 
    predictionCol="prediction"
)

rmse = evaluator.evaluate(predictions)
print(f"RMSE: {rmse}")

RMSE: 0.9180976315123347


In [9]:
user_id = 123
users_df = spark.createDataFrame([(user_id,)], ["user_id"])

recommendations = model.recommendForUserSubset(users_df, 5)

recommendations.show(truncate=False)

+-------+---------------------------------------------------------------------------------------------+
|user_id|recommendations                                                                              |
+-------+---------------------------------------------------------------------------------------------+
|123    |[{2236, 5.4559803}, {9024, 5.079625}, {7580, 5.0629807}, {9243, 4.9658628}, {2149, 4.939191}]|
+-------+---------------------------------------------------------------------------------------------+



In [13]:
spark.stop()

🎯 핵심 개념: ALS는 사람 간의 패턴 유사성을 학습

ALS는 실제로 책의 제목이나 장르, 저자 같은 콘텐츠 정보는 전혀 모릅니다.

대신에 ALS는 이런 식으로 작동해요:

**"A 사용자가 좋아한 책과 B 사용자가 좋아한 책이 많이 겹친다면, A와 B는 취향이 비슷하다.**

그러면 B가 좋아했지만 A가 아직 안 본 책도 A가 좋아할 확률이 높다!"

즉, **행동 데이터(= 평점)** 만 보고

"비슷한 사용자"와 "비슷한 책"을 수치적으로 추정해서 추천하는 겁니다.